In [1]:
# Game Ping-Pong

"""
    >Importação de bibliotecas necessárias / tkinter é um biblioteca utilizada para criação de interfaces gráficas
    >Definição de Variáveis Globais
        level: define-se o nível do jogo (no caso a proporção entre o canvas e a barra)
        length: variável auxiliar para definir o tamanho da barra
        count: define a pontuação alcançada
        lost: define quando se perde o jogo
"""
from tkinter import *
import random
import time

level = int(input('Qual nível você gostaria de jogar? 1/2/3/4/5 \n'))
length = 500 / level

"""
    >Inicialização do interpretador tcl/tk e criação da janela raiz/principal
    >Classe Tk(Misc, Wm)
    >Método title (herdado do Wm): seta título da ferramenta / title = wm_title(self, string=None)
    >Método resizable (herdado do Wm): instrui o gerenciador se dimensões podem ser redimensionadas em LARGURA e ALTURA. Ambos valores são booleanos / resizable = wm_resizable(self, width=None, height=None)
    >Método wm_attributes: esse comando retorna ou define comandos específicos da plataforma. O valor '-topmost' obtém ou define se esta é uma janela superior (exibida acima de todas as outras janelas no Windows)  / wm_attributes(self, *args)
    >Mpetodo mainloop(n=0) - chamada do loop principal do Tk
"""
root = Tk()
root.title("Ping Pong")
root.resizable(0, 0)
root.wm_attributes("-topmost", -1)

"""
    >Ferramenta Canvas que mostra elementos gráficos como linhas e textos
    >Classe Canvas(Widget, XView, YView)
    >Recurso highlightthickness do método __init__;
    >Método pack: empacota a ferramentaa da classe herdada / pack = pack_configure(self, cnf={}, **kw)
"""
canvas = Canvas(root, width=800, height=600, bd=0, highlightthickness=0)
canvas.pack()

root.update() ### Método update (herdado do Misc da Tk): entra no 'Event loop' até que todos os eventos pendentes tenham sido processados pelo Tcl

count = 0
lost = False

"""
    >Classe Bola
    >Definição de Parâmetros
        canvas: refere-se à instância canvas criada anteriormente;
        Barra: refere-se à instância Barra criada;
        color: cor da Bola, definida quando da criação da instância da Bola;
    >Definição de Variáveis Locais
        starts_x: variável auxiliar que define o sentido do movimento inicial da bola
        pos: variável que registra as coordenadas da bola conforme se move
        Barra_pos: variável que define a posição da barra na Canvas
    >Definição de Métodos
        draw: faz a bola se mover atraves do canvas
"""
class Bola:
    def __init__(self, canvas, Barra, color):
        self.canvas = canvas
        self.Barra = Barra
        self.id = canvas.create_oval(0, 0, 15, 15, fill=color) ### create_oval(*args, **kw) - Create oval with coordinates x1, y1, x2, y2
        self.canvas.move(self.id, 245, 200) ### move(self, *args) - Move an item TAGORID given in ARGS / arguments 245 and 200 refers to the initial coordinates

        starts_x = [-3, -2, -1, 1, 2, 3]
        random.shuffle(starts_x) ### shuffles the starts_x list

        self.x = starts_x[0] ### directions the ball starts (x axis)
        self.y = -3 ### directions the ball starts (y axis)

        self.canvas_height = self.canvas.winfo_height() ### winfo_height() - return height of this widget;
        self.canvas_width = self.canvas.winfo_width() ### winfo_width() - return the width of this widget;

    def draw(self):
        self.canvas.move(self.id, self.x, self.y)

        pos = self.canvas.coords(self.id) ### coords(*args) - Return a list of coordinates for the item given in ARGS

        ### quando a bola bate no teto (extremo superior) do canvas
        if pos[1] <= 0:
            self.y = 3

        ### ???
        if pos[3] >= self.canvas_height:
            self.y = -3

        ### quando a bola bate na lateral esquerda (canto esquerdo) do canvas
        if pos[0] <= 0:
            self.x = 3

        ### quando a bola bate na lateral direita (canto direita) do canvas
        if pos[2] >= self.canvas_width:
            self.x = -3

        self.Barra_pos = self.canvas.coords(self.Barra.id)

        ### quando a bola bate na barra, volta para cima
        if pos[2] >= self.Barra_pos[0] and pos[0] <= self.Barra_pos[2]:
            if pos[3] >= self.Barra_pos[1] and pos[3] <= self.Barra_pos[3]:
                self.y = -3
                global count ### global keyword allows you to modify the variable outside of the current scope
                count += 1
                score()

        ### mantém em movimento a bola e chama a rotina de quando se perde o jogo
        if pos[3] <= self.canvas_height:
            self.canvas.after(10, self.draw) ### after(ms, func=None, *args) - Call function once after given time - MS specifies the time in milliseconds. FUNC gives the function which shall be called. Additional parameters are given as parameters to the function call. Return identifier to cancel scheduling with after_cancel
        else:
            game_over()
            global lost
            lost = True

"""
    >Classe Barra
    >Definição de Parâmetros
        canvas: refere-se à instância canvas criada anteriormente;
        color: cor da Barra, definida quando da criação da instância da Barra;
    >Definição de Variáveis Locais
        x: variável que define as coordenadas da barra
        pos: variável que registra as coordenadas da barra conforme se move
    >Definição de Métodos
        draw: responsável pelo movimento da barra através do canvas
        move_left: leitura do comando de mover à esquerda
        move_right: leitura do comando de mover à direita
"""
class Barra:
    def __init__(self, canvas, color):
        self.canvas = canvas
        self.id = canvas.create_rectangle(0, 0, length, 10, fill=color) ### create_rectangle(*args, **kw) - Create rectangle with coordinates x1, y1, x2, y2
        self.canvas.move(self.id, 200, 400)

        self.x = 0

        self.canvas_width = self.canvas.winfo_width()

        self.canvas.bind_all("<KeyPress-Left>", self.move_left) ### bind_all(sequence=None, func=None, add=None) - Bind to all widgets at an event SEQUENCE a call to function FUNC. An additional boolean parameter ADD specifies whether FUNC will be called additionally to the other bound function or whether it will replace the previous function
        self.canvas.bind_all("<KeyPress-Right>", self.move_right)

    def draw(self):
        self.canvas.move(self.id, self.x, 0)

        self.pos = self.canvas.coords(self.id)

        ### limita à esquerda até onde a barra vai no canvas
        if self.pos[0] <= 0:
            self.x = 0

        ### limita à direita até onde a barra vai no canvas
        if self.pos[2] >= self.canvas_width:
            self.x = 0

        global lost

        ### faz com que a barra se movimente
        if lost == False:
            self.canvas.after(10, self.draw)

    def move_left(self, event):
        if self.pos[0] >= 0:
            self.x = -3

    def move_right(self, event):
        if self.pos[2] <= self.canvas_width:
            self.x = 3

"""
    >Função start_game: inicialização das variáveis e dos métodos draw
    >Função score: quantidade de pontos feitos
    >Função game_over: mensagem de fim do jogo
"""
def start_game(event):
    global lost, count
    lost = False
    count = 0
    score()
    canvas.itemconfig(game, text=" ") ### itemconfigure(tagOrId, cnf=None, **kw) - Configure resources of an item TAGORID. The values for resoures are specified as keyboard arguments. To get an overview about the allowed keyword arguments call the method without arguments.

    time.sleep(1)
    Barra.draw()
    Bola.draw()

def score():
    canvas.itemconfig(score_now, text="Pontos: " + str(count))

def game_over():
    canvas.itemconfig(game, text="Game Over!")

### Criações de Instâncias e chamada dos objetos
Barra = Barra(canvas, "orange")
Bola = Bola(canvas, Barra, "purple")

score_now = canvas.create_text(430, 20, text="Pontos: " + str(count), fill="green", font=("Arial", 16)) ### create_text(*args, **kw) - Create text with coordinates x1, y1
game = canvas.create_text(400, 300, text=" ", fill="red", font=("Arial", 40))

canvas.bind_all("<Button-1>", start_game)

root.mainloop()

Qual nível você gostaria de jogar? 1/2/3/4/5 
2
